# ansicolor

> Store color with ansi escape code

- Fore: foreground
- Back: background
- style: font style

In [ ]:
#| default_exp ansicolor

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.utils import *

In [ ]:
#| export
from colortextpy.color import enum, _EnumMeta, Color


class _Style(_EnumMeta):
    reset_all =        (enum.auto(), '00', True)
    end =              (enum.auto(), '00', True)
    bold =             (enum.auto(), '01', True)
    dim =              (enum.auto(), '02', False) # Not widely supported.
    faint =            (enum.auto(), '02', False) # Not widely supported.
    normal_intensity = (enum.auto(), '22', True)
    italic =           (enum.auto(), '03', False) # Not widely supported.
    no_italic =        (enum.auto(), '23', True)
    underline =        (enum.auto(), '04', True)
    no_underline =     (enum.auto(), '24', True)
    blink =            (enum.auto(), '05', True)
    slow_blink =       (enum.auto(), '05', True)
    rapid_blink =      (enum.auto(), '06', False) # # Not widely supported.
    no_blink =         (enum.auto(), '25', True)
    invert =           (enum.auto(), '07', True)
    no_invert =        (enum.auto(), '27', True)
    hidden =           (enum.auto(), '08', False) # Not widely supported.
    no_hidden =        (enum.auto(), '28', False)
    cross_out =        (enum.auto(), '09', False) # Not widely supported.
    strike =           (enum.auto(), '09', False) # Not widely supported.
    no_strike =        (enum.auto(), '29', False)
    
    def __init__(self, value, n, widely):
        self._value_ = value
        self._n = n
        self._widely = widely
        
    @property
    def n(self):
        return self._n
    
    @property
    def widely(self):
        return self._widely
    
    @classmethod
    @property
    def available(cls):
        return tuple(c.name for c in cls if c.widely)
    
class _AnsiColor:
    templates = dict(
        style = '\033[{0}m',
        fore = '\033[38;2;{0};{1};{2}m',
        back = '\033[48;2;{0};{1};{2}m',       
    )    
    
    def __init__(self, name='fore'):
        '''
        name: style, fore, back
        '''        
        self.name = name
        template = self.get_template(name=name)
        
        if name in ('fore', 'back'):
            for c in Color.available:
                setattr(self, c, template.format(*Color[c].rgb))
                
            reset = '39' if name == 'fore' else '49'
            self.reset = f'\033[{reset}m'
            self.reset_all = '\033[00m'
            self.available = Color.available
        else:
            for c in _Style:
                #if c.widely:
                if 1:
                    setattr(self, c.name, template.format(c.n))
            self.available = _Style.available
            
    def __repr__(self):
        return f'<AnsiColor: \'{self.name.upper()}\'>'
    
    def __getitem__(self, name):
        return getattr(self, name, '') if name is not None else ''
    
    def __getattr__(self, name):
        return self.__dict__[name] if name in self.__dict__ else ''
    
    def __contains__(self, item):
        return item in self.available
    
    def get_template(self, name='style'):
        '''
        name: style, fore, back
        '''
        return self.templates[name]
    
Fore = _AnsiColor('fore')
Back = _AnsiColor('back')
Style = _AnsiColor('style')

In [ ]:
show_doc(_AnsiColor)

---

[source](https://github.com/susuky/colortextpy/blob/main/colortextpy/ansicolor.py#L51){target="_blank" style="float:right; font-size:smaller"}

### _AnsiColor

>      _AnsiColor (name='fore')

name: style, fore, back

Some ansi escape code example:

In [ ]:
Fore.aliceblue, Back.blue, Style.bold

('\x1b', '\x1b', '\x1b')

Available color in `Fore`: 

In [ ]:
#| output false
Fore.available

('aliceblue',
 'antiquewhite',
 'aqua',
 'aquamarine',
 'azure',
 'bisque',
 'black',
 'blanchedalmond',
 'blue',
 'blueviolet',
 'brown',
 'burlywood',
 'cadetblue',
 'chartreuse',
 'chocolate',
 'coral',
 'cornflowerblue',
 'cornsilk',
 'crimson',
 'cyan',
 'c',
 'dark_blue',
 'dark_cyan',
 'dark_goldenrod',
 'dark_gray',
 'dark_green',
 'dark_khaki',
 'dark_magenta',
 'dark_olivegreen',
 'dark_orange',
 'dark_orchid',
 'dark_red',
 'dark_salmon',
 'dark_seagreen',
 'dark_slateblue',
 'dark_slategray',
 'dark_turquoise',
 'dark_violet',
 'deep_pink',
 'deep_skyblue',
 'dim_gray',
 'dodgerblue',
 'firebrick',
 'floralwhite',
 'forestgreen',
 'fuchsia',
 'gainsboro',
 'ghostwhite',
 'gold',
 'goldenrod',
 'gray',
 'green',
 'g',
 'greenyellow',
 'honeydew',
 'hotpink',
 'indianred',
 'indigo',
 'ivory',
 'khaki',
 'lavender',
 'lavenderblush',
 'lawngreen',
 'lemonchiffon',
 'light_blue',
 'light_coral',
 'light_cyan',
 'light_goldenrodyellow',
 'light_green',
 'light_grey',
 'light_pi

In [ ]:
#| echo: false
print('\033[48;2;50;50;50m')
for i, c in enumerate(Fore.available):
    end = '\n' if (i+1) % 5 == 0 else '\t'
    print(f'{Fore[c]}{c:21s}{Fore.reset}', end=end)
print(Back.reset_all)


aliceblue            	antiquewhite         	aqua                 	aquamarine           	azure                
bisque               	black                	blanchedalmond       	blue                 	blueviolet           
brown                	burlywood            	cadetblue            	chartreuse           	chocolate            
coral                	cornflowerblue       	cornsilk             	crimson              	cyan                 
c                    	dark_blue            	dark_cyan            	dark_goldenrod       	dark_gray            
dark_green           	dark_khaki           	dark_magenta         	dark_olivegreen      	dark_orange          
dark_orchid          	dark_red             	dark_salmon          	dark_seagreen        	dark_slateblue       
dark_slategray       	dark_turquoise       	dark_violet          	deep_pink            	deep_skyblue         
dim_gray             	dodgerblue           	firebrick            	floralwhite          	forestgreen          
fuchsia  

Available color in `Back`:

In [ ]:
#| output false
Back.available

('aliceblue',
 'antiquewhite',
 'aqua',
 'aquamarine',
 'azure',
 'bisque',
 'black',
 'blanchedalmond',
 'blue',
 'blueviolet',
 'brown',
 'burlywood',
 'cadetblue',
 'chartreuse',
 'chocolate',
 'coral',
 'cornflowerblue',
 'cornsilk',
 'crimson',
 'cyan',
 'c',
 'dark_blue',
 'dark_cyan',
 'dark_goldenrod',
 'dark_gray',
 'dark_green',
 'dark_khaki',
 'dark_magenta',
 'dark_olivegreen',
 'dark_orange',
 'dark_orchid',
 'dark_red',
 'dark_salmon',
 'dark_seagreen',
 'dark_slateblue',
 'dark_slategray',
 'dark_turquoise',
 'dark_violet',
 'deep_pink',
 'deep_skyblue',
 'dim_gray',
 'dodgerblue',
 'firebrick',
 'floralwhite',
 'forestgreen',
 'fuchsia',
 'gainsboro',
 'ghostwhite',
 'gold',
 'goldenrod',
 'gray',
 'green',
 'g',
 'greenyellow',
 'honeydew',
 'hotpink',
 'indianred',
 'indigo',
 'ivory',
 'khaki',
 'lavender',
 'lavenderblush',
 'lawngreen',
 'lemonchiffon',
 'light_blue',
 'light_coral',
 'light_cyan',
 'light_goldenrodyellow',
 'light_green',
 'light_grey',
 'light_pi

In [ ]:
#| echo: false
for i, c in enumerate(Back.available):
    end = '\n' if (i+1) % 5 == 0 else '\t'
    print(f'{Back[c]}{c:21s}{Back.reset}', end=end)

aliceblue            	antiquewhite         	aqua                 	aquamarine           	azure                
bisque               	black                	blanchedalmond       	blue                 	blueviolet           
brown                	burlywood            	cadetblue            	chartreuse           	chocolate            
coral                	cornflowerblue       	cornsilk             	crimson              	cyan                 
c                    	dark_blue            	dark_cyan            	dark_goldenrod       	dark_gray            
dark_green           	dark_khaki           	dark_magenta         	dark_olivegreen      	dark_orange          
dark_orchid          	dark_red             	dark_salmon          	dark_seagreen        	dark_slateblue       
dark_slategray       	dark_turquoise       	dark_violet          	deep_pink            	deep_skyblue         
dim_gray             	dodgerblue           	firebrick            	floralwhite          	forestgreen          
fuchsia   

In [ ]:
#| output false
Style.available

('reset_all',
 'end',
 'bold',
 'normal_intensity',
 'no_italic',
 'underline',
 'no_underline',
 'blink',
 'slow_blink',
 'no_blink',
 'invert',
 'no_invert')

In [ ]:
#| echo: false
for i, c in enumerate(('bold', 'dim', 'normal_intensity', 'underline', 'blink', 'invert')):
    print(f'{Style[c]}{c}{Style.end}', end=' | ')

bold | dim | normal_intensity | underline | blink | invert | 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()